<a href="https://colab.research.google.com/github/siren7075/hackathon2024/blob/main/thresholds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get Anomalies for SPOT, sort by date and group by currency , then get percentile changes and their thresholds

In [7]:
import pandas as pd
from scipy import stats

# Load the CSV data
df = pd.read_csv('fx_rates_spot_forward_MID_3mo_fromUSD.csv')

# Filter the DataFrame for fxType "SPOT"
spot_df = df[df['fxType'] == 'SPOT']

# Group by currency and sort by effectiveDate
spot_df = spot_df.sort_values(by=['toCurrencyCode', 'effectiveDate'])

# Prepare a DataFrame to store anomalies and thresholds
anomalies = pd.DataFrame()

# Calculate percentage changes and thresholds for each currency group
for currency, group in spot_df.groupby('toCurrencyCode'):
    group['percentage_change'] = group['spotRate'].pct_change() * 100

    # Calculate mean and standard deviation of percentage changes
    mean_change = group['percentage_change'].mean()
    std_dev_change = group['percentage_change'].std()

    # Define thresholds using Z-score method
    k = 2  # Adjust k for strictness
    lower_threshold = mean_change - k * std_dev_change
    upper_threshold = mean_change + k * std_dev_change

    # Identify anomalies based on the thresholds
    group['anomaly'] = ((group['percentage_change'] < lower_threshold) |
                        (group['percentage_change'] > upper_threshold))

    # Store the thresholds in the group DataFrame
    group['lower_threshold'] = lower_threshold
    group['upper_threshold'] = upper_threshold

    # Append anomalies to the anomalies DataFrame
    anomalies = pd.concat([anomalies, group[group['anomaly']]])

# Display the results with thresholds
print("Dynamic Thresholds and Anomalies:")
print("Currencies with Anomalies and Their Effective Dates:")
anomaly_list = anomalies[['toCurrencyCode', 'effectiveDate']].drop_duplicates()
for index, row in anomaly_list.iterrows():
    print(f"Currency: {row['toCurrencyCode']}, Effective Date: {row['effectiveDate']}")
print(anomalies[['effectiveDate', 'toCurrencyCode', 'spotRate', 'percentage_change', 'lower_threshold', 'upper_threshold']])


Dynamic Thresholds and Anomalies:
Currencies with Anomalies and Their Effective Dates:
Currency: AED, Effective Date: 2024-09-18
Currency: AED, Effective Date: 2024-10-04
Currency: AED, Effective Date: 2024-10-07
Currency: AED, Effective Date: 2024-10-08
Currency: AED, Effective Date: 2024-10-09
Currency: AED, Effective Date: 2024-10-10
Currency: AED, Effective Date: 2024-10-11
Currency: AED, Effective Date: 2024-10-14
Currency: AED, Effective Date: 2024-10-15
Currency: AED, Effective Date: 2024-10-18
Currency: AED, Effective Date: 2024-10-21
Currency: ALL, Effective Date: 2024-11-06
Currency: ALL, Effective Date: 2024-11-22
Currency: ALL, Effective Date: 2024-11-25
Currency: AOA, Effective Date: 2024-10-02
Currency: AOA, Effective Date: 2024-10-03
Currency: ARS, Effective Date: 2024-10-14
Currency: ARS, Effective Date: 2024-10-15
Currency: ARS, Effective Date: 2024-12-09
Currency: ATS, Effective Date: 2024-11-06
Currency: ATS, Effective Date: 2024-11-07
Currency: ATS, Effective Date: 